In [1]:
#! pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\supnova\AppData\Local\Temp\pip-install-6ykx2xzk\pytorch_8f96f9362afe4f5684c9f335a671142a\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\supnova\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra

In [87]:
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

import matplotlib.pyplot as plt


In [48]:
df_b_g = pd.read_csv('./p_b_growth_data.csv')
df_b_e = pd.read_csv('./p_b_ev_data.csv')


df_c_g = pd.read_csv('./p_c_growth_data.csv')
df_c_e = pd.read_csv('./p_c_ev_data.csv')

df_d_g = pd.read_csv('./p_d_growth_data.csv')
df_d_e = pd.read_csv('./p_d_ev_data.csv')
df_e = pd.read_csv('./p_e_growth_data.csv')
np.set_printoptions(precision = 3, suppress = True)


In [49]:
train_x = df_b_e.loc[3:,'supplyEC':].to_numpy()
train_x = train_x.reshape(29,1,6)
print(train_x)
print(np.shape(train_x))





[[[   0.966    5.999 1021.772   81.508   15.869  116.58 ]]

 [[   1.       6.    1022.634   81.252   13.338  112.797]]

 [[   1.02     5.964 1022.623   96.81    14.017  104.153]]

 [[   0.842    6.072  968.744   88.85    16.22    88.791]]

 [[   0.93     6.067 1021.985   86.565   13.912   73.803]]

 [[   1.       6.    1022.464   87.817   12.487   69.489]]

 [[   0.868    5.881  431.556   72.86    13.113   87.262]]

 [[   0.919    5.675  443.675   70.096   11.676   57.994]]

 [[   1.       5.999  441.701   87.736   12.61    64.118]]

 [[   1.       6.     369.502   95.794   12.201   33.891]]

 [[   1.       6.     407.886   90.979   12.171   70.555]]

 [[   0.       6.     476.146   93.1     12.733   84.293]]

 [[   0.662    5.885  439.828   94.561   10.912   51.296]]

 [[   0.866    6.     476.647   93.409   10.832   63.071]]

 [[   0.725    5.998  489.454   95.83    10.453   47.067]]

 [[   1.       6.     476.963   92.043   11.018   77.267]]

 [[   1.       5.998  437.946   91.656  

In [50]:
test_x = df_d_e.loc[2:,'supplyEC':].to_numpy()
test_x = test_x.reshape(27,1,6)
print(test_x)
print(np.shape(test_x))


[[[   0.588    5.994   24.215    3.543    0.822    8.141]]

 [[   0.56     5.998    0.       0.       0.       0.   ]]

 [[   0.222    5.998  187.082   31.238    6.743   32.452]]

 [[   0.607    5.991  460.591   71.573   16.044   77.296]]

 [[   0.398    5.994  672.304   73.564   15.904   69.889]]

 [[   0.378    5.994  855.773   81.428   14.955   74.829]]

 [[   0.492    5.987  759.121   77.459   16.847   74.098]]

 [[   0.714    5.996  850.066   87.693   14.923   82.063]]

 [[   0.82     5.995 1751.61    81.629   14.425   71.038]]

 [[   0.398    6.    1010.359   84.097   13.833   84.122]]

 [[   0.368    6.004 1115.107   81.813   14.827   78.122]]

 [[   0.292    5.999 1529.133   78.721   15.042   75.103]]

 [[   0.348    5.999 1413.102   83.123   13.545   76.5  ]]

 [[   0.697    6.002 1827.624   76.945   14.982   70.28 ]]

 [[   0.915    5.992  680.803   77.206   15.379   72.414]]

 [[   0.551    5.954 1319.99    78.411   14.407   79.2  ]]

 [[   0.574    5.769  715.941   76.828  

In [51]:
ty_early_g = df_b_g.loc[0,'초장(mm)':].to_numpy()
train_y = df_b_g.loc[:,'초장(mm)':].diff(axis=0).loc[1:].to_numpy()

print(ty_early_g)
print(train_y)
print(np.shape(train_y))

[263.889  80.444  71.222 149.      4.556  12.256   0.      0.      0.   ]
[[ 19.333  10.889  10.889  -4.444   0.667   3.927   0.      0.      0.   ]
 [  4.222   5.778   7.333 -18.      0.333   2.852   0.      0.      0.   ]
 [  4.     -0.556   0.     -6.778   0.778   1.808   0.      0.      0.444]
 [ -2.222   0.333   1.222  -5.222   0.667   1.356   0.667   1.222   0.556]
 [-41.     -4.556  -5.556  -5.667  -1.222   1.973   0.889   3.444   0.   ]
 [  7.556  -0.667  -0.889  16.889   0.889   1.413   0.444   1.778   0.   ]
 [  6.889  -2.333  -0.444  -2.778  -0.333   1.596  -0.778   3.333   0.   ]
 [  8.222  -1.667  -5.667   7.556   0.333   0.156   0.222   1.111   0.   ]
 [-16.667  -1.444  -3.667   0.      0.444  -0.469   0.111   0.556   0.   ]
 [ 17.528  -2.722  -4.944  30.194   0.389  -1.279  -1.556  -2.944   0.   ]
 [ 23.028   3.278   7.5    13.472   0.167   0.864   0.     -0.833   0.   ]
 [-12.889  -5.333  -5.667 -10.     -0.556  -0.362   0.     -1.444   0.   ]
 [  0.667   4.333   1.889 

In [52]:
test_y_early_g =  df_d_g.loc[0,'초장(mm)':].to_numpy()
test_y = df_d_g.loc[:,'초장(mm)':].diff(axis=0).loc[1:].to_numpy()

print(test_y_early_g)
print(test_y)
print(np.shape(test_y))

[164.222  68.111  56.333  51.933   4.222  13.264   0.      0.      0.   ]
[[ 12.333  20.778  22.      6.733  -0.222   4.397   0.      0.      0.   ]
 [ -0.778   1.222   6.778   8.889   0.889   1.358   0.222   0.111   0.444]
 [ 52.333   9.111   3.111  17.111   1.      3.439   2.333   2.667   0.556]
 [ 25.111  11.222   3.444  40.778   0.222  -0.694  -0.111   3.      0.   ]
 [ 33.556   1.333  -1.111   8.778  -1.778   0.498  -0.667   4.      0.   ]
 [  4.889  -4.778  -4.667  15.778   1.      0.919  -1.667   0.444   0.   ]
 [  2.556   0.111  -1.444  -6.      0.111   0.227  -0.111  -0.222   0.   ]
 [  8.778  -0.111  -1.889   2.222   0.667   2.004   0.     -1.      0.   ]
 [  3.889  -5.778   1.444  13.778  -0.111  -0.386   0.     -1.444   0.   ]
 [ -4.556  -0.222  -8.111   1.444   0.     -0.069   0.     -1.111   0.   ]
 [  2.222  -6.889  -6.333  12.      0.444  -1.869   0.     -2.667   0.   ]
 [ -1.222  10.333   7.222 -16.444  -0.222   1.552   0.     -2.778   0.   ]
 [  0.667 -19.444 -11.889 

In [92]:
Batch_size = 28
#learning_rate = 0.1
Epochs = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device: %s"%(device))


device: cpu


In [93]:
model = Sequential()
model.add(LSTM(units=20, activation='relu', input_shape=(1,6)))
model.add(RepeatVector(9))
model.add(Dropout(0.2))
model.add(LSTM(units=20, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))



In [94]:
model.compile(optimizer='adam', loss='mean_absolute_error')
model.fit(train_x, train_y, epochs=Epochs , batch_size=Batch_size)


Epoch 1/100
2/2 [==============================] - 2s 5ms/step - loss: 7.7273
Epoch 2/100
2/2 [==============================] - 0s 6ms/step - loss: 7.0482
Epoch 3/100
2/2 [==============================] - 0s 5ms/step - loss: 7.0914
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 6.3890
Epoch 5/100
2/2 [==============================] - 0s 5ms/step - loss: 6.2516
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 5.7655
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 5.4640
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 5.1488
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 5.0086
Epoch 10/100
2/2 [==============================] - 0s 5ms/step - loss: 5.1902
Epoch 11/100
2/2 [==============================] - 0s 4ms/step - loss: 4.9489
Epoch 12/100
2/2 [==============================] - 0s 4ms/step - loss: 4.8225
Epoch 13/100
2/2 [==============================] - 0s 4ms/st

In [95]:
preb_y = model.predict(test_x)
np.set_printoptions(precision = 3, suppress = True)
preb_y = preb_y.reshape(len(preb_y),9)

#for i in range(1, len(preb_y)):
#    preb_y[i] = 

preb_y = np.insert(preb_y, 0,test_y_early_g,axis=0)


for i in range(1, len(preb_y)):
    for j in range(9):
        if preb_y[i-1][j] + preb_y[i][j] < 0 :
            if i == len(preb_y)-1:
                pass
            else:
                preb_y[i+1][j] = preb_y[i+1][j]+preb_y[i][j]
            preb_y[i][j] = 0
        else:
            preb_y[i][j] = preb_y[i][j] + preb_y[i-1][j]

for i in range(len(preb_y)):
    for j in range(9):
        if j == 4 or j == 6 or j == 7 or j == 8:
            preb_y[i][j] = math.ceil(preb_y[i][j])
print(preb_y)
#plt.figure()
#plt.plot(test_y, color='red', label='실제값')
#plt.plot(preb_y, color='blue', label='예측값')
#plt.legend()
#plt.show()

1/1 [==============================] - 0s 257ms/step
[[164.222  68.111  56.333  51.933   5.     13.264   0.      0.      0.   ]
 [164.337  68.272  56.529  52.155   5.     13.519   1.      1.      1.   ]
 [164.399  68.339  56.6    52.229   5.     13.596   1.      1.      1.   ]
 [164.456  68.398  56.661  52.291   5.     13.661   1.      1.      1.   ]
 [164.519  68.467  56.734  52.367   5.     13.741   1.      1.      1.   ]
 [164.52   68.4    56.616  52.215   5.     13.563   1.      1.      1.   ]
 [164.814  68.069  56.042  51.725   5.     13.507   1.      1.      1.   ]
 [164.848  67.924  55.776  51.399   4.     13.228   1.      1.      1.   ]
 [164.925  67.727  55.419  50.992   4.     12.951   1.      1.      1.   ]
 [166.134  66.789  54.857  50.965   4.     13.508   1.      1.      2.   ]
 [166.566  66.39   54.207  50.502   4.     13.595   1.      2.      2.   ]
 [167.309  65.79   53.46   50.193   4.     13.958   2.      2.      3.   ]
 [168.36   64.958  52.782  50.083   5.     14.4

In [96]:
model.save("./vvsb_preTest_model.model")

INFO:tensorflow:Assets written to: ./vvsb_preTest_model.model\assets


: 